In [1]:
!pip install pycimg
!pip install opencv-python

from pycimg import CImg
import numpy as np
from scipy.ndimage import rotate
import cv2
from IPython.display import display, Image
import math




In [2]:
#Variable global per fer-m'ho fàcil
clicked_coordinates = []

# Callback function for mouse click event
def on_click(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        clicked_coordinates.append((x, y))

path = 'images/brain/'

In [3]:

def distance(p1, p2):
    return math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)

def triangle_size(C1, C2, C3):
    
    A=distance(C1,C2)
    B=distance(C1,C3)
    C=distance(C2,C3)
    
    sizes = (A,B,C)
    return list(sizes)


def angle_between_points(p1, p2):
    
    p3 = (p1[0],p2[1])
    
    a = distance(p2, p3)
    b = distance(p1, p3)
    c = distance(p1, p2)
    
    cos_A = (c**2 + b**2 - a**2) / (2 * c * b)
    angle_radians = math.acos(cos_A)
    angle_degrees = math.degrees(angle_radians)
    if(p2[0]>p1[0]):
        angle_degrees=180-angle_degrees
    return angle_degrees


def triangle_center(point1, point2, point3):
    # Calculate the average x-coordinate
    center_x = (point1[0] + point2[0] + point3[0]) / 3
    # Calculate the average y-coordinate
    center_y = (point1[1] + point2[1] + point3[1]) / 3
    return center_x, center_y


def find_zoom(img_1_pos, img_2_pos):
    dist1 = distance(img_1_pos[0],img_1_pos[1])
    dist2 = distance(img_2_pos[0],img_2_pos[1])

    zoom = float(dist1/dist2)
    
    return zoom
    

In [4]:

# Load image using OpenCV
image_path = path+'Brain_87.tiff'
image = cv2.imread(image_path)

# Display image using OpenCV
cv2.namedWindow('image')
cv2.imshow('image', image)

# Set the mouse callback function for the OpenCV window
cv2.setMouseCallback('image', on_click)

cv2.waitKey(0)

cv2.destroyAllWindows()


In [11]:
image_path_1 = path+'dibuix_1.png'
image_path_2 = path+'dibuix_2_zoomedout.png'

image_1 = cv2.imread(image_path_1)
image_2 = cv2.imread(image_path_2)

proc = False

#Empty the saved coordinates
clicked_coordinates=[]

# Display image using OpenCV
cv2.namedWindow('Primera Imatge')
cv2.imshow('Primera Imatge', image_1)

# Set the mouse callback function for the OpenCV window
cv2.setMouseCallback('Primera Imatge', on_click)

cv2.waitKey(0)

cv2.destroyAllWindows()

coord_1=(0,0)
coord_2=(0,0)
coord_3=(0,0)

#Guardaré les tres últimes només abans del enter fent servir un stack
if len(clicked_coordinates)>=3:
    coord_1=clicked_coordinates.pop()
    coord_2=clicked_coordinates.pop()
    coord_3=clicked_coordinates.pop()
    
    positions_1 = [coord_1,coord_2]
    
    sizes = triangle_size(coord_1,coord_2,coord_3)
    print(sorted(sizes,reverse=True))

    reference_degree = (angle_between_points(coord_2,coord_1))

    clicked_coordinates=[]

        # Display image using OpenCV
    cv2.namedWindow('Segona imatge')
    cv2.imshow('Segona imatge', image_2)

    # Set the mouse callback function for the OpenCV window
    cv2.setMouseCallback('Segona imatge', on_click)

    cv2.waitKey(0)

    cv2.destroyAllWindows()

    coord_1=(0,0)
    coord_2=(0,0)
    coord_3=(0,0)

    #Guardaré les tres últimes només abans del enter fent servir un stack
    if len(clicked_coordinates)>=3:
        coord_1=clicked_coordinates.pop()
        coord_2=clicked_coordinates.pop()
        coord_3=clicked_coordinates.pop()

        positions_2 = [coord_1,coord_2]

        sizes = triangle_size(coord_1,coord_2,coord_3)
        print(sorted(sizes,reverse=True))

        rotated_degree = (angle_between_points(coord_2,coord_1))

        difference =(rotated_degree-reference_degree)
        
        zoom_factor = find_zoom(positions_1, positions_2)

    else:
        print('2nd image: Click at least 3 times.')
    
else:
    print('1st image: Click at least 3 times.')


[361.4484748895754, 353.8318809830454, 332.73713348527843]
[133.41664064126334, 129.49517365523704, 123.08127396155761]


In [12]:
height, width = image_2.shape[:2]
print(difference)
# Define the rotation angle (in degrees)
angle = difference
# Calculate the rotation matrix
rotation_matrix = cv2.getRotationMatrix2D((coord_2), angle, 1)

# Perform the rotation
rotated_image = cv2.warpAffine(image_2, rotation_matrix, (width, height))


new_width = int(rotated_image.shape[1] * zoom_factor)
new_height = int(rotated_image.shape[0] * zoom_factor)




zoomed_image = cv2.resize(rotated_image, (new_width, new_height))


-15.210699954710051


In [13]:
# Display the original and rotated images (optional)
cv2.imshow('Original Image', image_1)
cv2.imshow('Rotated Image', zoomed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()